In [0]:
import keras,os
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Activation, Dropout
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

### **Some data augmentation technique.**

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   width_shift_range=[-200,200],
                                   height_shift_range=0.5,
                                   zoom_range=[0.5,1.0],
                                   brightness_range=[0.2,1.0],
                                   rotation_range=90,
                                   horizontal_flip = True)

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### **Dataset is uploaded to google drive and directory path is taken from there.**

In [0]:
train_generator = ImageDataGenerator(preprocessing_function = keras.applications.vgg16.preprocess_input).flow_from_directory(
        "/content/drive/My Drive/dataset/trian",
        target_size=(224, 224))

Found 596 images belonging to 2 classes.


In [0]:
test_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/dataset/test',
        target_size = (224, 224))

Found 50 images belonging to 2 classes.


### **Transfer learning using VGG16 model**

In [0]:
vgg_model = VGG16(input_shape=(224, 224, 3), weights='imagenet')

553467904/553467096 [==============================] - 7s 0us/step


In [0]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### **We not taking entire architecture from vgg we are some part of it. So, we creating our own model**


In [0]:
model = Sequential()

In [0]:
#Taking till 10th layer

for layer in vgg_model.layers[:11]:
    model.add(layer)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [0]:
#Freezing first 3 layers

for layer in model.layers[:-9]:
  layer.trainable = False

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(2, activation="softmax"))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [0]:
model.compile(
  loss='categorical_crossentropy',
  optimizer="Adam",
  metrics=['accuracy']
)

In [0]:
history = model.fit_generator(train_generator,
                    validation_data=test_generator,
                    epochs=10,
                    steps_per_epoch=74,
                    validation_steps=10
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
74/74 [==============================] - 564s 8s/step - loss: 384.5068 - accuracy: 0.9751 - val_loss: 52495.4453 - val_accuracy: 0.4600
Epoch 2/10
74/74 [==============================] - 26s 349ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 61222.6797 - val_accuracy: 0.4520
Epoch 3/10
74/74 [==============================] - 26s 350ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 59502.8555 - val_accuracy: 0.4600
Epoch 4/10
74/74 [==============================] - 26s 351ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 58402.9258 - val_accuracy: 0.4640
Epoch 5/10
74/74 [==============================] - 26s 352ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 60702.9492 - val_accuracy: 0.4600
Epoch 6/10
74/74 [==============================] - 26s 352ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 61360.6875 - val_accuracy: 0.4640
Epoch 7/10
74/74 [====

In [0]:
model.save("model.h5")